In [1]:
import pandas as pd
import numpy as np
from scipy import stats
%matplotlib inline
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
import pandas as pd
import statsmodels.api as sm
from IPython.display import Image
from sklearn import linear_model, datasets
from sklearn.metrics import mean_squared_error
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import pylab 
import scipy.stats as stats

/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
df=pd.read_csv("zomato.csv",encoding = "ISO-8859-1",low_memory=False)
print(df.columns.values)
df.head()

['Restaurant ID' 'Restaurant Name' 'Country Code' 'City' 'Address'
 'Locality' 'Locality Verbose' 'Longitude' 'Latitude' 'Cuisines'
 'Average Cost for two' 'Currency' 'Has Table booking'
 'Has Online delivery' 'Is delivering now' 'Switch to order menu'
 'Price range' 'Aggregate rating' 'Rating color' 'Rating text' 'Votes']


,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


In [98]:
df_reduced = df[["Country Code","City", "Cuisines", "Average Cost for two", 
                 "Currency", "Has Table booking", "Has Online delivery", 
                 "Is delivering now", "Aggregate rating", 
                 "Rating color", "Votes"
                ]]
df_reduced.rename(columns={"Average Cost for two": "AvgCost"}, inplace=True)
df_reduced.rename(columns={"Has Table booking": "HasTableBooking"}, inplace=True)
df_reduced.rename(columns={"Has Online delivery": "HasOnlineDelivery"}, inplace=True)
df_reduced.rename(columns={"Is delivering now": "IsDeliveringNow"}, inplace=True)
df_reduced.rename(columns={"Country Code": "CountryCode"}, inplace=True)
df_reduced.rename(columns={"Aggregate rating":"AggregateRating"}, inplace=True)
df_reduced.rename(columns={"Rating color": "RatingColor"}, inplace=True)
df_reduced.RatingColor.unique()

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



array(['Dark Green', 'Green', 'Yellow', 'Orange', 'White', 'Red'], dtype=object)

In [99]:
cuisines = df_reduced.Cuisines.str.split(',', expand=True).stack().str.strip().reset_index(level=1, drop=True)
df1 = pd.DataFrame(data=cuisines, columns=["Cuisines"])
df_reduced_cuisines = df_reduced.drop(['Cuisines'], axis=1).join(df1).reset_index(drop=True)
df_reduced_cuisines.groupby("Cuisines").mean()


,CountryCode,AvgCost,AggregateRating,Votes
Cuisines,,,,
Afghani,46.714286,498.214286,1.971429,57.928571
African,74.625000,907.500000,3.525000,627.500000
American,87.938462,678.294872,3.661538,469.530769
Andhra,1.000000,794.000000,3.870000,594.600000
Arabian,46.178571,773.571429,3.385714,219.642857
Argentine,215.500000,45.000000,4.300000,488.000000
Armenian,1.000000,366.666667,1.300000,55.333333
Asian,38.605150,5750.708155,3.815021,447.652361
Asian Fusion,148.000000,80.000000,3.850000,73.500000


In [102]:
# This is the statsmodel method
model = ols("AvgCost ~ CountryCode + HasTableBooking + Votes + Cuisines", 
            data=df_reduced_cuisines).fit()
print(model.summary())
print("R-squared is %.4f" % model.rsquared)
print("R-squared adjusted is %.4f" % model.rsquared_adj)

                            OLS Regression Results                            
Dep. Variable:                AvgCost   R-squared:                       0.208
Model:                            OLS   Adj. R-squared:                  0.202
Method:                 Least Squares   F-statistic:                     34.88
Date:                Sat, 07 Apr 2018   Prob (F-statistic):               0.00
Time:                        11:16:46   Log-Likelihood:            -2.1733e+05
No. Observations:               19710   AIC:                         4.350e+05
Df Residuals:                   19562   BIC:                         4.361e+05
Df Model:                         147                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept     

In [101]:
feature_data = df_reduced_cuisines.drop("AvgCost", axis=1)
target_data = df_reduced_cuisines["AvgCost"]

feature_data["HasTableBooking"] = pd.get_dummies(feature_data["HasTableBooking"])["Yes"]
feature_data["HasOnlineDelivery"] = pd.get_dummies(feature_data["HasOnlineDelivery"])["Yes"]
feature_data["IsDeliveringNow"] = pd.get_dummies(feature_data["IsDeliveringNow"])["Yes"]
feature_data["RatingColor-DarkGreen"] = pd.get_dummies(feature_data["RatingColor"])['Dark Green']
feature_data["RatingColor-Green"] = pd.get_dummies(feature_data["RatingColor"])['Green']

reg = linear_model.LinearRegression()  # this is the sklearn method
reg.fit(feature_data, target_data)
print(reg.coef_)
print("Intercept is %.4f" % reg.intercept_)

# We can get the r-squared value using the score function
r2 = reg.score(feature_data, target_data)
# To get the adjusted r-squared, we use a slightly different formula:
# r2-adj = 1 - [((1-r2) * (n-1)) / (n - p - 1)]
r2_adj = 1 - (1-r2)*(len(target_data)-1) / (len(target_data) - feature_data.shape[1]-1)

print("R-squared is %.4f" % r2)
print("Adjusted R-squared is %.4f" % r2_adj)

# Feature Correlation

corr_matrix = feature_data.corr()
corr_matrix

ValueError: could not convert string to float: 'Cafe'

In [80]:
trace = go.Heatmap(z=np.array(corr_matrix), x=corr_matrix.columns, y=corr_matrix.columns)
data=[trace]
iplot(data)